In [31]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("ReadJSON").getOrCreate()

# Read JSON file into DataFrame
review = spark.read.json(r"C:\MS DS\RG_Winter25\Capstone 1 DSCI 591\data\review-Pennsylvania.json\review-Pennsylvania.json")
meta = spark.read.json(r"C:\MS DS\RG_Winter25\Capstone 1 DSCI 591\data\meta-Pennsylvania.json\meta-Pennsylvania.json")


# Show DataFrame
meta.show()


+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+
|                MISC|             address|avg_rating|            category|         description|             gmap_id|               hours|          latitude|         longitude|                name|num_of_reviews|price|    relative_results|               state|                 url|
+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+
|{NULL, NULL, NULL...|Primo Pizza & Sub...|       4.5|    [Pizza delivery]|                NULL|0x89c46d5e4554eae...|[[Thursday, 11AM–...|        40.68217

In [32]:
review.show()


+--------------------+------------------+----+------+----+--------------------+-------------+--------------------+
|             gmap_id|              name|pics|rating|resp|                text|         time|             user_id|
+--------------------+------------------+----+------+----+--------------------+-------------+--------------------+
|0x89c46d5e4554eae...|   Jennifer Cordón|NULL|     5|NULL|Just had the Germ...|1630529977304|10241275264630097...|
|0x89c46d5e4554eae...|     Luis Gonzalez|NULL|     5|NULL|Great food, and g...|1630544172046|10330038687874311...|
|0x89c46d5e4554eae...|     Jasson Cordon|NULL|     5|NULL|Good food and fai...|1631012728526|10471584331703059...|
|0x89c46d5e4554eae...|     Sharon Mendez|NULL|     5|NULL|Very good service...|1630460639875|10772877940368786...|
|0x89c46d5e4554eae...|  Salvador Fuentes|NULL|     5|NULL|Great food and se...|1630435490430|10879752391481534...|
|0x89c46d5e4554eae...|     Kevin Fuentes|NULL|     5|NULL|Good service and ...|1

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, lower, regexp_replace, trim, udf
from pyspark.sql.types import StringType, FloatType, TimestampType
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import json

# # Initialize Spark Session
# spark = SparkSession.builder \
#     .appName("ReadJSON") \
#     .config("spark.driver.memory", "4g") \
#     .config("spark.executor.memory", "4g") \
#     .getOrCreate()

# # Read JSON files into PySpark DataFrames
# reviews = spark.read.json(r"C:\MS DS\RG_Winter25\Capstone 1 DSCI 591\data\review-Pennsylvania.json")
# meta = spark.read.json(r"C:\MS DS\RG_Winter25\Capstone 1 DSCI 591\data\meta-Pennsylvania.json")

# Display schema
review.printSchema()
meta.printSchema()


# Rename 'name' column in reviews and metadata to avoid conflicts
reviews = review.withColumnRenamed("name", "reviewer_name")
metadata = meta.withColumnRenamed("name", "business_name")

# Merge the datasets on 'gmap_id'
merged_df = reviews.join(metadata, on="gmap_id", how="inner")

from pyspark.sql.functions import col, when, to_json

from pyspark.sql.functions import col, when, to_json

# Ensure `resp` is a STRUCT before converting to JSON
if "resp" in merged_df.columns:
    resp_dtype = dict(merged_df.dtypes)["resp"]  # Get column type
    if "struct" in resp_dtype.lower():
        merged_df = merged_df.withColumn("resp", to_json(col("resp")))



from pyspark.sql.functions import col, when

misc_df = merged_df.select("gmap_id", "MISC", "hours", "relative_results")

# 2) Drop the three columns from the main dataframe
merged_df = merged_df.drop("MISC", "hours", "relative_results")

# Define column groups
text_columns = ["text", "description", "resp"]  # Fill with empty string ""







root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)

root
 |-- MISC: struct (nullable = true)
 |    |-- Accessibility: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Activities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Amenities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Atmosphere: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Crowd: array (nullable =

In [34]:

merged_df = merged_df.withColumn("time", col("time").cast(TimestampType()))


In [35]:

# Convert numeric columns to appropriate types
numeric_fields = ["rating", "user_id", "avg_rating", "num_of_reviews", "latitude", "longitude"]
for field in numeric_fields:
    merged_df = merged_df.withColumn(field, col(field).cast(FloatType()))


In [24]:

# Load NLTK stopwords
nltk.download("punkt")
nltk.download("stopwords")
# Function to clean text (Lowercase, remove punctuation, remove stopwords)
def clean_text(text):
    text = text.lower()
    text = regexp_replace(text, "[^a-zA-Z0-9 ]", "")  # Remove punctuation
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

# Register UDF for text cleaning
clean_text_udf = udf(clean_text, StringType())

# Apply text cleaning UDF
# merged_df = merged_df.withColumn("clean_text", clean_text_udf(col("text")))


# Drop unwanted columns
columns_to_drop = ["category", "relative_results"]
merged_df = merged_df.drop(*columns_to_drop)

# Display final DataFrame
merged_df.show(5)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Py4JJavaError: An error occurred while calling o795.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnecti

In [ ]:

# Function to clean text (Lowercase, remove punctuation, remove stopwords)
def clean_text(text):
    text = text.lower()
    text = regexp_replace(text, "[^a-zA-Z0-9 ]", "")  # Remove punctuation
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

# Register UDF for text cleaning
clean_text_udf = udf(clean_text, StringType())

# Apply text cleaning UDF
merged_df = merged_df.withColumn("clean_text", clean_text_udf(col("text")))

# Rename columns
merged_df = merged_df.withColumnRenamed("name_x", "customer_name") \
                     .withColumnRenamed("name_y", "business_name")

# Drop unwanted columns
columns_to_drop = ["category", "hours", "MISC", "relative_results"]
merged_df = merged_df.drop(*columns_to_drop)

# Display final DataFrame
merged_df.show(5)



In [25]:

# Define PostgreSQL connection parameters
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "Capstone"
DB_USER = "postgres"
DB_PASSWORD = "Google#13"
TABLE_NAME = "main"

jdbc_url = f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}"
properties = {"user": DB_USER, "password": DB_PASSWORD, "driver": "org.postgresql.Driver"}

# Write data to PostgreSQL in chunks (Parallel processing)
try:
    merged_df.write \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", TABLE_NAME) \
        .option("user", DB_USER) \
        .option("password", DB_PASSWORD) \
        .option("driver", "org.postgresql.Driver") \
        .mode("append") \
        .save()
    
    print(f"Data successfully dumped to table '{TABLE_NAME}' in PostgreSQL!")
except Exception as e:
    print("Error:", e)

# Stop Spark Session
spark.stop()


Error: An error occurred while calling o804.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:258)
	at org.apache

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Example") \
    .getOrCreate()

# Make sure spark isn't None
print(spark)


In [36]:
pandas_df = merged_df.toPandas()

Py4JJavaError: An error occurred while calling o934.collectToPython.
: org.apache.spark.SparkException: Job 6 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1251)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1251)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:3087)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$stop$3(DAGScheduler.scala:2973)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2973)
	at org.apache.spark.SparkContext.$anonfun$stop$12(SparkContext.scala:2258

In [28]:
merged_df

DataFrame[gmap_id: string, reviewer_name: string, pics: array<struct<url:array<string>>>, rating: float, resp: string, text: string, time: timestamp, user_id: float, address: string, avg_rating: float, description: string, latitude: float, longitude: float, business_name: string, num_of_reviews: float, price: string, state: string, url: string]